<a href="https://colab.research.google.com/github/vutl/Automatic-Speech-Recognition/blob/main/ASR_Project_Vietnamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
kynthesis_vivos_vietnamese_speech_corpus_for_asr_path = kagglehub.dataset_download('kynthesis/vivos-vietnamese-speech-corpus-for-asr')

print('Data source import complete.')


In [ ]:
# Cài đặt các thư viện cần thiết
!pip install --upgrade transformers datasets jiwer librosa evaluate matplotlib torchaudio seaborn

In [ ]:
# Import các thư viện
import os
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import wave
import contextlib
import torch
import re
import json
import evaluate  # Thay thế load_metric bằng evaluate
from datasets import load_dataset, Audio, Dataset
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    Wav2Vec2CTCTokenizer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
%matplotlib inline

In [ ]:
# Kiểm tra thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Đường dẫn đến dataset VIVOS
train_audio_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/train/waves'
train_prompts_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/train/prompts.txt'
train_genders_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/train/genders.txt'

test_audio_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/test/waves'
test_prompts_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/test/prompts.txt'
test_genders_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/test/genders.txt'

In [ ]:
# Hàm để đọc file prompts.txt và trả về DataFrame
def load_prompts(prompts_path):
    transcripts = []
    with open(prompts_path, 'r', encoding='utf-8') as f:
        for line in f:
            id, text = line.strip().split(' ', 1)
            transcripts.append({'id': id, 'text': text.lower()})
    return pd.DataFrame(transcripts)

# Tạo DataFrame cho tập train và test
train_transcripts = load_prompts(train_prompts_path)
test_transcripts = load_prompts(test_prompts_path)

In [ ]:
# Thêm đường dẫn âm thanh vào DataFrame
def get_audio_path(audio_base_path, audio_id):
    speaker = audio_id.split('_')[0]
    return os.path.join(audio_base_path, speaker, audio_id + '.wav')

train_transcripts['audio'] = train_transcripts['id'].apply(lambda x: get_audio_path(train_audio_path, x))
test_transcripts['audio'] = test_transcripts['id'].apply(lambda x: get_audio_path(test_audio_path, x))

In [ ]:
# Loại bỏ các ký tự đặc biệt và chuyển văn bản về chữ thường
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"“%‘”�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

# Áp dụng hàm tiền xử lý dữ liệu
train_transcripts = train_transcripts.apply(remove_special_characters, axis=1)
test_transcripts = test_transcripts.apply(remove_special_characters, axis=1)

# Kiểm tra một số mẫu text sau khi tiền xử lý
print("=== Một Số Mẫu Text từ Tập Train Sau Khi Tiền Xử Lý ===")
for i in range(5):
    sample = train_transcripts.iloc[i]['text']
    print(f"Mẫu {i+1}: {sample}")
    print("-" * 50)

print("=== Một Số Mẫu Text từ Tập Test Sau Khi Tiền Xử Lý ===")
for i in range(5):
    sample = test_transcripts.iloc[i]['text']
    print(f"Mẫu {i+1}: {sample}")
    print("-" * 50)

In [ ]:
# vocab_dict = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7, "h": 8, "i": 9, "j": 10, "k": 11, "l": 12, "m": 13, "n": 14, "o": 15, "p": 16, "q": 17, "r": 18, "s": 19, "t": 20, "u": 21, "v": 22, "w": 23, "x": 24, "y": 25, "z": 26, "à": 27, "á": 28, "â": 29, "ã": 30, "è": 31, "é": 32, "ê": 33, "ì": 34, "í": 35, "ò": 36, "ó": 37, "ô": 38, "õ": 39, "ù": 40, "ú": 41, "ý": 42, "ă": 43, "đ": 44, "ĩ": 45, "ũ": 46, "ơ": 47, "ư": 48, "ạ": 49, "ả": 50, "ấ": 51, "ầ": 52, "ẩ": 53, "ẫ": 54, "ậ": 55, "ắ": 56, "ằ": 57, "ẳ": 58, "ẵ": 59, "ặ": 60, "ẹ": 61, "ẻ": 62, "ẽ": 63, "ế": 64, "ề": 65, "ể": 66, "ễ": 67, "ệ": 68, "ỉ": 69, "ị": 70, "ọ": 71, "ỏ": 72, "ố": 73, "ồ": 74, "ổ": 75, "ỗ": 76, "ộ": 77, "ớ": 78, "ờ": 79, "ở": 80, "ỡ": 81, "ợ": 82, "ụ": 83, "ủ": 84, "ứ": 85, "ừ": 86, "ử": 87, "ữ": 88, "ự": 89, "ỳ": 90, "ỵ": 91, "ỷ": 92, "ỹ": 93, "|": 0, "<bos>": 94, "<eos>": 95, "<unk>": 96, "<pad>": 97}

In [ ]:
# Tải processor và model từ mô hình pre-trained

model = Wav2Vec2ForCTC.from_pretrained("CuongLD/wav2vec2-large-xlsr-vietnamese")
model.to(device)

vi_tokenizer = Wav2Vec2CTCTokenizer(config.vi_vocab_file, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

processor = Wav2Vec2Processor.from_pretrained("CuongLD/wav2vec2-large-xlsr-vietnamese", tokenizer = vi_tokenizer)

In [ ]:
# 10. Kiểm tra vocab của tokenizer
print("=== Một Số Tokens Từ Tokenizer ===")
vocab = processor.tokenizer.get_vocab()
sorted_vocab = sorted(vocab.items(), key=lambda item: item[1])

for token, id in sorted_vocab[:100]:  # In ra 100 token đầu tiên
    print(f"Token: {token} - ID: {id}")

# 11. Kiểm tra mã hóa một số mẫu văn bản thủ công với processor
print("=== Kiểm Tra Mã Hóa Thủ Công Với Processor ===")
for i, text in enumerate(sample_texts):
    encoding = processor.tokenizer(text, padding=False, truncation=False, add_special_tokens=False)
    input_ids = encoding['input_ids']
    tokens = processor.tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
    print(f"Mẫu {i+1}:")
    print(f"Text: {text}")
    print(f"Encoded IDs: {input_ids}")
    print(f"Tokens: {' '.join(tokens)}")
    print("-" * 50)

# 12. Kiểm tra mã hóa các ký tự riêng biệt
print("=== Kiểm Tra Mã Hóa Các Ký Tự Riêng Biệt ===")
for i, text in enumerate(sample_texts):
    print(f"Mẫu {i+1}: {text}")
    for char in text:
        encoding = processor.tokenizer(char, add_special_tokens=False)
        input_ids = encoding['input_ids']
        tokens = processor.tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        print(f"Ký tự: '{char}' - ID: {input_ids} - Token: {tokens}")
    print("-" * 50)

# 13. Kiểm tra cấu hình của tokenizer
print("=== Cấu Hình Tokenizer ===")
print(processor.tokenizer.config)

In [ ]:
print(processor.tokenizer)

In [ ]:
# Chuyển đổi DataFrame thành Dataset của Hugging Face
train_dataset = Dataset.from_pandas(train_transcripts)
test_dataset = Dataset.from_pandas(test_transcripts)

# Chuyển cột 'audio' thành kiểu Audio
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# Hàm tiền xử lý dữ liệu
def prepare_dataset(batch):
    # Xử lý âm thanh
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    # Xử lý văn bản với tokenizer đã chuẩn hóa
    encoding = processor.tokenizer(batch["text"], padding=False, truncation=False, add_special_tokens=False)
    batch["labels"] = encoding["input_ids"]
    return batch

# Áp dụng hàm tiền xử lý dữ liệu
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

In [ ]:
# Một số mẫu văn bản tiếng Việt
sample_texts = [
    "khách sạn",
    "chỉ bằng cách luôn nỗ lực thì cuối cùng bạn mới được đền đáp",
    "trong số các quốc gia công nghiệp phát triển",
    "anh đã nhìn thấy trong những lải nhải dông dài của nhu",
    "khủng hoảng môi trường cần được ngăn chặn"
]

print("=== Kiểm Tra Mã Hóa Thủ Công Với Tokenizer Độc Lập ===")
for i, text in enumerate(sample_texts):
    encoding = processor.tokenizer(text, padding=False, truncation=False, add_special_tokens=False)
    input_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
    print(f"Mẫu {i+1}:")
    print(f"Text: {text}")
    print(f"Encoded IDs: {input_ids}")
    print(f"Tokens: {' '.join(tokens)}")
    print("-" * 50)

In [ ]:
#Kiểm tra một số mẫu labels từ tập train và test
print("=== Mẫu Labels từ Tập Train ===")
for i in range(5):
    sample = train_dataset[i]
    labels = sample['labels']
    # Chuyển đổi IDs sang tokens, bỏ qua các token padding
    label_tokens = processor.tokenizer.convert_ids_to_tokens(labels, skip_special_tokens=True)
    print(f"Mẫu {i+1}:")
    print(f"Labels IDs: {labels}")
    print(f"Labels Tokens: {' '.join(label_tokens)}")
    print("-" * 50)

print("=== Mẫu Labels từ Tập Test ===")
for i in range(5):
    sample = test_dataset[i]
    labels = sample['labels']
    # Chuyển đổi IDs sang tokens, bỏ qua các token padding
    label_tokens = processor.tokenizer.convert_ids_to_tokens(labels, skip_special_tokens=True)
    print(f"Mẫu {i+1}:")
    print(f"Labels IDs: {labels}")
    print(f"Labels Tokens: {' '.join(label_tokens)}")
    print("-" * 50)

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Tách inputs và labels
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Padding inputs
        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Padding labels
        with self.processor.as_target_processor():
            labels_batch = self.processor.tokenizer.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # Thay thế giá trị padding bằng -100
        labels = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)

        batch["labels"] = labels

        return batch

# Khởi tạo Data Collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# Định nghĩa hàm tính WER
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Giải mã dự đoán
    pred_str = processor.batch_decode(pred_ids)

    # Giải mã nhãn
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    # Tính WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
from transformers import EarlyStoppingCallback

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./wav2vec2-vivos",
    group_by_length=True,
    per_device_train_batch_size=4,  # Giảm batch size nếu cần
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=10,  # Có thể điều chỉnh
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=5e-5,  # Giảm learning rate từ 1e-4 xuống 5e-5
    warmup_steps=500,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

# Callback cho Early Stopping
callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,  # Truyền toàn bộ processor
    callbacks=callbacks,
)

In [ ]:
# Bắt đầu huấn luyện
trainer.train()

# Lưu mô hình cuối cùng
trainer.save_model("./wav2vec2-vivos")
# Lưu processor
processor.save_pretrained("./wav2vec2-vivos")

In [ ]:
# Tải lại processor và mô hình đã huấn luyện
processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-vivos")
model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-vivos")
model.to(device)

# Chuyển mô hình sang chế độ đánh giá
model.eval()

# In ra một số mẫu text từ tập train và test sau khi tiền xử lý
print("=== Mẫu Text từ Tập Train ===")
for i in range(5):
    sample = train_transcripts.iloc[i]['text']
    print(f"Mẫu {i+1}: {sample}")
    print("-" * 50)

print("=== Mẫu Text từ Tập Test ===")
for i in range(5):
    sample = test_transcripts.iloc[i]['text']
    print(f"Mẫu {i+1}: {sample}")
    print("-" * 50)

# Kiểm tra lại vocab của tokenizer sau khi huấn luyện
print("=== Vocab Sau Khi Huấn Luyện ===")
for i, (token, id) in enumerate(processor.tokenizer.get_vocab().items()):
    if i < 100:
        print(f"Token: {token} - ID: {id}")
    else:
        break

# Chọn một file âm thanh từ tập test
audio_file = test_transcripts['audio'].iloc[0]
print(f"Đang thử nghiệm với file âm thanh: {audio_file}")

# Đọc file âm thanh
audio_input, sampling_rate = librosa.load(audio_file, sr=16000)

# Tiền xử lý âm thanh
input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values.to(device)

# Dự đoán
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

print("Kết quả nhận dạng giọng nói:")
print(transcription)

In [ ]:
# Đánh giá mô hình trên tập test
def evaluate_model(dataset):
    model.eval()
    predictions = []
    references = []

    for sample in dataset:
        audio = sample["input_values"]
        label = sample["labels"]

        # Chuyển input_values về tensor và di chuyển vào thiết bị
        input_tensor = torch.tensor(audio).unsqueeze(0).to(device)

        # Dự đoán
        with torch.no_grad():
            logits = model(input_tensor).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        predictions.append(transcription)

        # Giải mã nhãn
        label_ids = label
        label_ids = [id for id in label_ids if id != -100]
        label_str = processor.tokenizer.decode(label_ids, skip_special_tokens=True)
        references.append(label_str)

    wer = wer_metric.compute(predictions=predictions, references=references)
    print(f"WER trên tập test: {wer * 100:.2f} %")

# Chạy đánh giá
evaluate_model(test_dataset)

In [ ]:
# # Lưu vocabulary thành file JSON
# with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
#     json.dump(vocab_dict, vocab_file, ensure_ascii=False)

# # Khởi tạo tokenizer
# tokenizer = Wav2Vec2CTCTokenizer("vocab.json",
#                                  unk_token="<unk>",
#                                  pad_token="<pad>",
#                                  word_delimiter_token="|")

In [ ]:
# from collections import Counter

# # Tạo danh sách tất cả các từ trong tập dữ liệu
# all_words = ' '.join(full_dataset['text']).split()
# word_counts = Counter(all_words)

# # Số lượng từ duy nhất
# num_unique_words = len(word_counts)
# print(f"Số lượng từ duy nhất trong tập dữ liệu: {num_unique_words}")

# # 10 từ xuất hiện nhiều nhất
# most_common_words = word_counts.most_common(10)
# print("10 từ xuất hiện nhiều nhất:")
# for word, count in most_common_words:
#     print(f"{word}: {count}")

In [ ]:
# # Đọc file genders.txt và thêm thông tin giới tính
# def load_genders(genders_path):
#     genders = {}
#     with open(genders_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             speaker_id, gender = line.strip().split(' ')
#             genders[speaker_id] = gender
#     return genders

In [ ]:
# # Tải thông tin giới tính cho tập train và test
# train_genders = load_genders(train_genders_path)
# test_genders = load_genders(test_genders_path)

# # Thêm cột 'speaker' và 'gender' vào DataFrame
# train_transcripts['speaker'] = train_transcripts['id'].apply(lambda x: x.split('_')[0])
# test_transcripts['speaker'] = test_transcripts['id'].apply(lambda x: x.split('_')[0])

# train_transcripts['gender'] = train_transcripts['speaker'].map(train_genders)
# test_transcripts['gender'] = test_transcripts['speaker'].map(test_genders)

# # Thêm cột 'dataset' để phân biệt tập train và test
# train_transcripts['dataset'] = 'train'
# test_transcripts['dataset'] = 'test'

# # Kết hợp hai tập dữ liệu
# full_dataset = pd.concat([train_transcripts, test_transcripts], ignore_index=True)

# # Phân tích dữ liệu
# # a. Kiểm tra số lượng mẫu
# num_train_samples = len(train_transcripts)
# num_test_samples = len(test_transcripts)
# num_total_samples = len(full_dataset)

# print(f"Số lượng mẫu trong tập huấn luyện: {num_train_samples}")
# print(f"Số lượng mẫu trong tập kiểm tra: {num_test_samples}")
# print(f"Tổng số mẫu: {num_total_samples}")

# # b. Phân tích thời lượng âm thanh
# def get_audio_duration(audio_path):
#     with contextlib.closing(wave.open(audio_path,'r')) as f:
#         frames = f.getnframes()
#         rate = f.getframerate()
#         duration = frames / float(rate)
#         return duration

# # Thêm cột 'duration' vào DataFrame
# full_dataset['duration'] = full_dataset['audio'].apply(get_audio_duration)

# # Vẽ biểu đồ phân phối thời lượng âm thanh
# plt.figure(figsize=(10, 6))
# sns.histplot(full_dataset['duration'], bins=50, kde=True)
# plt.title('Phân phối thời lượng âm thanh')
# plt.xlabel('Thời lượng (giây)')
# plt.ylabel('Số lượng mẫu')
# plt.show()

# # c. Phân tích số lượng từ trong transcript
# def count_words(text):
#     return len(text.split())

# # Thêm cột 'num_words' vào DataFrame
# full_dataset['num_words'] = full_dataset['text'].apply(count_words)

# # Vẽ biểu đồ phân phối số lượng từ
# plt.figure(figsize=(10, 6))
# sns.histplot(full_dataset['num_words'], bins=30, kde=True)
# plt.title('Phân phối số lượng từ trong transcript')
# plt.xlabel('Số lượng từ')
# plt.ylabel('Số lượng mẫu')
# plt.show()

# # d. Phân tích sóng âm của một mẫu
# # Chọn một mẫu ngẫu nhiên
# sample = full_dataset.sample(1).iloc[0]
# audio_path_sample = sample['audio']
# text_sample = sample['text']

# # Đọc âm thanh
# audio_data_sample, sr_sample = librosa.load(audio_path_sample, sr=None)

# # Vẽ sóng âm
# plt.figure(figsize=(14, 5))
# librosa.display.waveshow(audio_data_sample, sr=sr_sample)
# plt.title(f"Sóng âm của mẫu: {sample['id']}")
# plt.xlabel('Thời gian (giây)')
# plt.ylabel('Biên độ')
# plt.show()

# print(f"Transcript: {text_sample}")

# # e. Phân tích phân bố giọng nam và nữ
# gender_counts = full_dataset['gender'].value_counts()
# print("Số lượng mẫu theo giới tính:")
# print(gender_counts)

# # Vẽ biểu đồ phân bố giới tính
# plt.figure(figsize=(6, 6))
# sns.countplot(x='gender', data=full_dataset)
# plt.title('Phân bố giới tính trong tập dữ liệu')
# plt.xlabel('Giới tính')
# plt.ylabel('Số lượng mẫu')
# plt.show()

# # Phân tích thời lượng âm thanh theo giới tính
# plt.figure(figsize=(10, 6))
# sns.kdeplot(data=full_dataset, x='duration', hue='gender', fill=True)
# plt.title('Phân phối thời lượng âm thanh theo giới tính')
# plt.xlabel('Thời lượng (giây)')
# plt.ylabel('Mật độ')
# plt.show()

In [ ]:
# # Lọc các mẫu có thời lượng từ 1 đến 7 giây
# filtered_dataset = full_dataset[(full_dataset['duration'] >= 1.0) & (full_dataset['duration'] <= 7.0)]
# print(f"Số lượng mẫu sau khi lọc: {len(filtered_dataset)}")

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x='duration', y='num_words', data=full_dataset)
# plt.title('Mối quan hệ giữa thời lượng âm thanh và số lượng từ')
# plt.xlabel('Thời lượng (giây)')
# plt.ylabel('Số lượng từ')
# plt.show()

In [ ]:
# # Tải thông tin giới tính cho tập train và test
# train_genders = load_genders(train_genders_path)
# test_genders = load_genders(test_genders_path)

# # Thêm cột 'speaker' và 'gender' vào DataFrame
# train_transcripts['speaker'] = train_transcripts['id'].apply(lambda x: x.split('_')[0])
# test_transcripts['speaker'] = test_transcripts['id'].apply(lambda x: x.split('_')[0])

# train_transcripts['gender'] = train_transcripts['speaker'].map(train_genders)
# test_transcripts['gender'] = test_transcripts['speaker'].map(test_genders)

# # Thêm cột 'dataset' để phân biệt tập train và test
# train_transcripts['dataset'] = 'train'
# test_transcripts['dataset'] = 'test'

# # Thêm cột 'duration' (thời lượng âm thanh)
# def get_audio_duration(audio_path):
#     with contextlib.closing(wave.open(audio_path,'r')) as f:
#         frames = f.getnframes()
#         rate = f.getframerate()
#         duration = frames / float(rate)
#         return duration

# train_transcripts['duration'] = train_transcripts['audio'].apply(get_audio_duration)
# test_transcripts['duration'] = test_transcripts['audio'].apply(get_audio_duration)

# # Thêm cột 'num_words' (số lượng từ trong transcript)
# def count_words(text):
#     return len(text.split())

# train_transcripts['num_words'] = train_transcripts['text'].apply(count_words)
# test_transcripts['num_words'] = test_transcripts['text'].apply(count_words)

# # So sánh tập train và test

# # 1. So sánh phân phối thời lượng âm thanh
# plt.figure(figsize=(12, 6))
# sns.kdeplot(data=train_transcripts, x='duration', label='Train', fill=True)
# sns.kdeplot(data=test_transcripts, x='duration', label='Test', fill=True)
# plt.title('Phân phối thời lượng âm thanh giữa tập Train và Test')
# plt.xlabel('Thời lượng (giây)')
# plt.ylabel('Mật độ')
# plt.legend()
# plt.show()

# # 2. So sánh phân phối số lượng từ trong transcript
# plt.figure(figsize=(12, 6))
# sns.kdeplot(data=train_transcripts, x='num_words', label='Train', fill=True)
# sns.kdeplot(data=test_transcripts, x='num_words', label='Test', fill=True)
# plt.title('Phân phối số lượng từ trong transcript giữa tập Train và Test')
# plt.xlabel('Số lượng từ')
# plt.ylabel('Mật độ')
# plt.legend()
# plt.show()

# # 3. So sánh phân bố giới tính
# gender_counts_train = train_transcripts['gender'].value_counts()
# gender_counts_test = test_transcripts['gender'].value_counts()

# gender_df = pd.DataFrame({
#     'train': gender_counts_train,
#     'test': gender_counts_test
# }).transpose()

# gender_df.plot(kind='bar', figsize=(8, 6))
# plt.title('Phân bố giới tính giữa tập Train và Test')
# plt.xlabel('Dataset')
# plt.ylabel('Số lượng mẫu')
# plt.legend(title='Giới tính')
# plt.show()

# # 4. So sánh danh sách speaker giữa tập train và test
# speakers_train = set(train_transcripts['speaker'])
# speakers_test = set(test_transcripts['speaker'])

# # Số lượng speaker trong mỗi tập
# print(f"Số lượng speaker trong tập Train: {len(speakers_train)}")
# print(f"Số lượng speaker trong tập Test: {len(speakers_test)}")

# # Kiểm tra xem có sự trùng lặp speaker giữa hai tập không
# common_speakers = speakers_train.intersection(speakers_test)
# print(f"Số lượng speaker có mặt ở cả hai tập: {len(common_speakers)}")

# # Nếu cần, liệt kê các speaker chung
# if len(common_speakers) > 0:
#     print("Danh sách speaker có mặt ở cả hai tập:")
#     print(common_speakers)